In [1]:
import pandas as pd
import re
import json
import ast

In [2]:
raw_df = pd.read_csv("../data/aws_heroes_raw.csv")
# df['fields'] = df['fields'].str.replace("HttpUrl\('(.*?)'\)", r"'\1'")
raw_df.head()

,id,dateCreated,dateUpdated,heroName,heroCategory,heroLocation,heroSinceDate,description,heroBioURL,heroImageURL,featureFlag,region,superpower,year,category
0,community-heroes#adeline-makokha,2025-06-11T19:09:16+0000,2025-06-24T17:50:56+0000,Adeline Makokha,AWS Community Hero,"Nairobi, Kenya",Hero since 2025,<p>Adeline transforms her curiosity about clou...,https://aws.amazon.com/developer/community/her...,https://d1.awsstatic.com/developer-center/adel...,NEW,"Europe, Middle East, & Africa","['Open Source Contributor', 'Meetup Organizer'...","['3000', '2025']",AWS Community Hero
1,community-heroes#david-victoria,2025-05-21T00:56:03+0000,2025-06-04T20:10:42+0000,David Victoria,AWS Community Hero,"Monterrey, Mexico",Hero since 2025,<p>David Victoria is a Senior Cloud Architect ...,https://aws.amazon.com/developer/community/her...,https://d1.awsstatic.com/developer-center/davi...,New,Latin America,"['Trainer / Educator', 'Meetup Organizer', 'Co...","['2025', '3000']",AWS Community Hero
2,community-heroes#christian-bonzelet,2025-05-21T00:46:24+0000,2025-06-24T17:51:44+0000,Christian Bonzelet,AWS DevTools Hero,"Cologne, Germany",Hero since 2025,<p>Christian&nbsp;is the founder of&nbsp;promp...,https://aws.amazon.com/developer/community/her...,https://d1.awsstatic.com/developer-center/chri...,New,"Europe, Middle East, & Africa","['Conference Speaker', 'Blogger']","['2025', '3000']",AWS DevTools Hero
3,community-heroes#nora-schoener,2025-05-21T00:51:39+0000,2025-06-06T19:39:10+0000,Nora Schöner,AWS DevTools Hero,"Erlangen, Germany",Hero since 2025,<p>Nora is a Senior Cloud Engineer with a long...,https://aws.amazon.com/developer/community/her...,https://d1.awsstatic.com/developer-center/nora...,New,"Europe, Middle East, & Africa","['Conference Speaker', 'Meetup Organizer', 'Bl...","['2025', '3000']",AWS DevTools Hero
4,community-heroes#badri-kesavan,2025-02-26T15:56:39+0000,2025-06-04T17:24:03+0000,Badri Narayanan Kesavan,AWS Community Hero,Singapore,Hero since 2025,<p>Badri is an Engineering Lead and Solutions ...,https://aws.amazon.com/developer/community/her...,https://d1.awsstatic.com/getting-started-guide...,NaN,Asia Pacific,"['Meetup Organizer', 'Conference Speaker']",['2025'],AWS Community Hero


In [3]:
# drop columns
cols_drop = ['dateCreated', 'dateUpdated', 'description', 'heroBioURL', 'year']
df = raw_df.drop(columns=cols_drop)
df.head()

,id,heroName,heroCategory,heroLocation,heroSinceDate,heroImageURL,featureFlag,region,superpower,category
0,community-heroes#adeline-makokha,Adeline Makokha,AWS Community Hero,"Nairobi, Kenya",Hero since 2025,https://d1.awsstatic.com/developer-center/adel...,NEW,"Europe, Middle East, & Africa","['Open Source Contributor', 'Meetup Organizer'...",AWS Community Hero
1,community-heroes#david-victoria,David Victoria,AWS Community Hero,"Monterrey, Mexico",Hero since 2025,https://d1.awsstatic.com/developer-center/davi...,New,Latin America,"['Trainer / Educator', 'Meetup Organizer', 'Co...",AWS Community Hero
2,community-heroes#christian-bonzelet,Christian Bonzelet,AWS DevTools Hero,"Cologne, Germany",Hero since 2025,https://d1.awsstatic.com/developer-center/chri...,New,"Europe, Middle East, & Africa","['Conference Speaker', 'Blogger']",AWS DevTools Hero
3,community-heroes#nora-schoener,Nora Schöner,AWS DevTools Hero,"Erlangen, Germany",Hero since 2025,https://d1.awsstatic.com/developer-center/nora...,New,"Europe, Middle East, & Africa","['Conference Speaker', 'Meetup Organizer', 'Bl...",AWS DevTools Hero
4,community-heroes#badri-kesavan,Badri Narayanan Kesavan,AWS Community Hero,Singapore,Hero since 2025,https://d1.awsstatic.com/getting-started-guide...,NaN,Asia Pacific,"['Meetup Organizer', 'Conference Speaker']",AWS Community Hero


## Heroes Countries and Regions

In [4]:
# hero country

df['heroCountry'] = df['heroLocation'].str.split(',').str[-1].str.strip()

In [5]:
# check uniqueness of country
print(df['heroCountry'].unique())

['Kenya' 'Mexico' 'Germany' 'Singapore' 'Brazil' 'United States'
 'Philippines' 'Israel' 'Australia' 'Poland' 'Saudi Arabia' 'MA'
 'Indonesia' 'Serbia' 'USA' 'Hong Kong' 'Spain' 'Finland' 'Norway'
 'Ontario Canada' 'Denmark' 'Vietnam' 'India' 'United Kingdom' 'Sweden'
 'Ukraine' 'New Zealand' 'Lebanon' 'Turkey' 'China' 'Belgium' 'Canada'
 'Perú' 'Colombia' 'Cameroon' 'Argentina' 'Bulgaria'
 'Bosnia and Herzegovina' 'Taiwan' 'Nigeria' 'Czech Republic' 'Luxembourg'
 'Portugal' 'Switzerland' 'Brussels. Belgium' 'Guatemala' 'Japan' 'Korea'
 'Italy' 'South Korea' 'France' 'Malaysia' 'Malta' 'Thailand' 'Greece'
 'Austria' 'Netherlands' 'Ireland' 'Hungary' 'Sri Lanka' 'The Netherlands'
 'Czechia' 'South Africa' 'United Arab Emirates' 'Romania']


In [6]:
# where is MA? turns out it is in the US
df[df['heroCountry'] == "MA"]

,id,heroName,heroCategory,heroLocation,heroSinceDate,heroImageURL,featureFlag,region,superpower,category,heroCountry
14,community-heroes#aj-stuyvenberg,AJ Stuyvenberg,AWS Serverless Hero,"Boston, MA",Hero since 2023,https://d1.awsstatic.com/getting-started-guide...,NaN,United States,['Conference Speaker'],AWS Serverless Hero,MA


In [7]:
# correct country mapping
renamed_countries = {
        "USA": "United States",
        "The Netherlands": "Netherlands",
        "Perú": "Peru",
        "Czechia": "Czech Republic",
        "Korea": "South Korea",
        "Brussels. Belgium": "Belgium",
        "Ontario Canada": "Canada",
        "MA": "United States",
        "Bosnia and Herzegovina": "Bosnia & Herzegovina"
    }
df['heroCountry'] = df['heroCountry'].replace(renamed_countries)

In [8]:
# countries and their regions
with open ('../data/country_regions.json', 'r') as file:
    cntry_reg_map = json.load(file)
    
df['heroRegion'] = df['heroCountry'].map(cntry_reg_map)
df.head(2)

,id,heroName,heroCategory,heroLocation,heroSinceDate,heroImageURL,featureFlag,region,superpower,category,heroCountry,heroRegion
0,community-heroes#adeline-makokha,Adeline Makokha,AWS Community Hero,"Nairobi, Kenya",Hero since 2025,https://d1.awsstatic.com/developer-center/adel...,NEW,"Europe, Middle East, & Africa","['Open Source Contributor', 'Meetup Organizer'...",AWS Community Hero,Kenya,"Europe, Middle East, & Africa"
1,community-heroes#david-victoria,David Victoria,AWS Community Hero,"Monterrey, Mexico",Hero since 2025,https://d1.awsstatic.com/developer-center/davi...,New,Latin America,"['Trainer / Educator', 'Meetup Organizer', 'Co...",AWS Community Hero,Mexico,North America


## Current and Former Heroes

In [9]:
# check the uniquesness of hero since date

df['heroSinceDate'].value_counts()

heroSinceDate
Hero since 2020           57
Hero since 2019           43
Hero since 2021           33
Hero since 2023           28
Hero since 2024           26
Hero since 2022           25
Hero since 2018           15
Hero since 2025            9
Hero since 2017            8
Hero since 2016            8
Hero since 2015            5
Hero since 2014            3
Hero from 2019 - 2022      2
Hero from 2020 - 2022      1
Hero from 2014 - 2022      1
Hero from 2015 - 2022      1
Hero from 2014 to 2022     1
Hero from 2021 - 2022      1
Name: count, dtype: int64

In [10]:
# split heroSinceDate to categorize active and inactive heroes
pattern = r'Hero (since|from) (\d{4})(?:.*?(\d{4}))?'

df[['statement_', 'heroSince', 'end_year']] = df['heroSinceDate'].str.extract(
    pattern,
    flags=re.IGNORECASE
)

# get status of heroes
df['status'] = df['statement_'].map({'since': 'Current Heros', 'from': 'Former Heroes'})
df.head(2)

,id,heroName,heroCategory,heroLocation,heroSinceDate,heroImageURL,featureFlag,region,superpower,category,heroCountry,heroRegion,statement_,heroSince,end_year,status
0,community-heroes#adeline-makokha,Adeline Makokha,AWS Community Hero,"Nairobi, Kenya",Hero since 2025,https://d1.awsstatic.com/developer-center/adel...,NEW,"Europe, Middle East, & Africa","['Open Source Contributor', 'Meetup Organizer'...",AWS Community Hero,Kenya,"Europe, Middle East, & Africa",since,2025,NaN,Current Heros
1,community-heroes#david-victoria,David Victoria,AWS Community Hero,"Monterrey, Mexico",Hero since 2025,https://d1.awsstatic.com/developer-center/davi...,New,Latin America,"['Trainer / Educator', 'Meetup Organizer', 'Co...",AWS Community Hero,Mexico,North America,since,2025,NaN,Current Heros


## Superpowers

In [11]:
# explode superpowers 
df['heroSuperpower'] = df['superpower'].str.replace(r"[\[\]']", "", regex=True)
df.head(2)

,id,heroName,heroCategory,heroLocation,heroSinceDate,heroImageURL,featureFlag,region,superpower,category,heroCountry,heroRegion,statement_,heroSince,end_year,status,heroSuperpower
0,community-heroes#adeline-makokha,Adeline Makokha,AWS Community Hero,"Nairobi, Kenya",Hero since 2025,https://d1.awsstatic.com/developer-center/adel...,NEW,"Europe, Middle East, & Africa","['Open Source Contributor', 'Meetup Organizer'...",AWS Community Hero,Kenya,"Europe, Middle East, & Africa",since,2025,NaN,Current Heros,"Open Source Contributor, Meetup Organizer, Blo..."
1,community-heroes#david-victoria,David Victoria,AWS Community Hero,"Monterrey, Mexico",Hero since 2025,https://d1.awsstatic.com/developer-center/davi...,New,Latin America,"['Trainer / Educator', 'Meetup Organizer', 'Co...",AWS Community Hero,Mexico,North America,since,2025,NaN,Current Heros,"Trainer / Educator, Meetup Organizer, Conferen..."


In [12]:
# Convert string representation of lists in 'superpower' to actual lists
df['superpower'] = df['superpower'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)

# Explode the 'superpower' column
df = df.explode('superpower')

# Reset index for clarity (optional)
df = df.reset_index(drop=True)
df.shape

(853, 17)

In [13]:
# check uniqueness of 
df['superpower'].value_counts()

superpower
Conference Speaker            202
Blogger                       190
Meetup Organizer              138
Technical Author              119
Open Source Contributor       104
Trainer / Educator             98
hero-alumni - {DO NOT USE}      1
Name: count, dtype: int64

In [14]:
# who the hero alumni
df[df['superpower']=='hero-alumni - {DO NOT USE}']

,id,heroName,heroCategory,heroLocation,heroSinceDate,heroImageURL,featureFlag,region,superpower,category,heroCountry,heroRegion,statement_,heroSince,end_year,status,heroSuperpower
712,community-heroes#shaoyi-li,Shaoyi Li,AWS Community Hero,"Shenzhen, China",Hero since 2024,https://d1.awsstatic.com/getting-started-guide...,NaN,Asia Pacific,hero-alumni - {DO NOT USE},AWS Community Hero,China,Asia Pacific,since,2024,NaN,Current Heros,"Blogger, Technical Author, Trainer / Educator,..."


In [15]:
df['category'].value_counts()

category
AWS Community Hero           375
AWS Serverless Hero          170
AWS Container Hero            90
AWS Machine Learning Hero     81
AWS Data Hero                 69
AWS DevTools Hero             48
AWS Security Hero             20
Name: count, dtype: int64

## Tests

In [16]:
# test: check the uniques of the statement_ column
# test output: should be since or from
df['statement_'].value_counts()

statement_
since    835
from      18
Name: count, dtype: int64

In [17]:
# test: check the uniques of heroSince column
# test: should be 2014 or greater 
df['heroSince'].value_counts()

heroSince
2020    183
2019    136
2021    121
2024     88
2022     86
2023     65
2018     57
2025     30
2017     27
2016     27
2015     22
2014     11
Name: count, dtype: int64

In [18]:
# test: check uniqueness of end_year column
# should be 2022 or greater
df['end_year'].value_counts()

end_year
2022    18
Name: count, dtype: int64

In [19]:
# test: check active and inactive users
# test output: if the output of the two are added we should get the dataset length
df['status'].value_counts()

status
Current Heros    835
Former Heroes     18
Name: count, dtype: int64

In [20]:
# test: check region uniqueness
# test output: only 4 regions
df['heroRegion'].value_counts()

heroRegion
Europe, Middle East, & Africa    362
Asia Pacific                     283
North America                    162
Latin America                     44
Name: count, dtype: int64

In [21]:
# check uniques of years
df['heroSince'].value_counts()

heroSince
2020    183
2019    136
2021    121
2024     88
2022     86
2023     65
2018     57
2025     30
2017     27
2016     27
2015     22
2014     11
Name: count, dtype: int64

## Drop Columns

In [22]:
# drop unnecessary columns
cols_drop_ = ['region', 'heroCategory', 'statement_', 'end_year']
df = df.drop(columns=cols_drop_)

In [23]:
df.head()

,id,heroName,heroLocation,heroSinceDate,heroImageURL,featureFlag,superpower,category,heroCountry,heroRegion,heroSince,status,heroSuperpower
0,community-heroes#adeline-makokha,Adeline Makokha,"Nairobi, Kenya",Hero since 2025,https://d1.awsstatic.com/developer-center/adel...,NEW,Open Source Contributor,AWS Community Hero,Kenya,"Europe, Middle East, & Africa",2025,Current Heros,"Open Source Contributor, Meetup Organizer, Blo..."
1,community-heroes#adeline-makokha,Adeline Makokha,"Nairobi, Kenya",Hero since 2025,https://d1.awsstatic.com/developer-center/adel...,NEW,Meetup Organizer,AWS Community Hero,Kenya,"Europe, Middle East, & Africa",2025,Current Heros,"Open Source Contributor, Meetup Organizer, Blo..."
2,community-heroes#adeline-makokha,Adeline Makokha,"Nairobi, Kenya",Hero since 2025,https://d1.awsstatic.com/developer-center/adel...,NEW,Blogger,AWS Community Hero,Kenya,"Europe, Middle East, & Africa",2025,Current Heros,"Open Source Contributor, Meetup Organizer, Blo..."
3,community-heroes#adeline-makokha,Adeline Makokha,"Nairobi, Kenya",Hero since 2025,https://d1.awsstatic.com/developer-center/adel...,NEW,Conference Speaker,AWS Community Hero,Kenya,"Europe, Middle East, & Africa",2025,Current Heros,"Open Source Contributor, Meetup Organizer, Blo..."
4,community-heroes#david-victoria,David Victoria,"Monterrey, Mexico",Hero since 2025,https://d1.awsstatic.com/developer-center/davi...,New,Trainer / Educator,AWS Community Hero,Mexico,North America,2025,Current Heros,"Trainer / Educator, Meetup Organizer, Conferen..."


## Save Data

In [24]:
# save the cleaned data
df.to_csv('../data/aws_heroes_cleaned.csv', index=False)